This kernel is inspired from https://www.kaggle.com/iafoss/256x256-images. You can use this data if some one need to work with 512x512 images.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os
from tqdm.notebook import tqdm
import zipfile
import tifffile as tiff

In [ ]:
train_csv = '../input/hubmap-kidney-segmentation/train.csv'
test_csv = '../input/hubmap-kidney-segmentation/train.csv'
train_folder = '../input/hubmap-kidney-segmentation/train'
test_folder = '../input/hubmap-kidney-segmentation/test'

## Basic EDA

Let look at images we have and let us see their size.

In [ ]:
images = [file for file in os.listdir(train_folder) if str(file).split('.')[1] == 'tiff']
for image_name in images:
    img = tiff.imread(os.path.join(train_folder,image_name))
    print(image_name,img.shape)

It is clear that some images have channels first and some other last. Also we need to squeeze some images as we go further.

In [ ]:
mask_df = pd.read_csv(train_csv)
# seting id as index as ot will be useful while iterating
# mask_df = mask_df.set_index('id')
mask_df.head()

In [ ]:

# https://www.kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
 
def rle2mask(mask_rle, shape=(1600,256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T


In [ ]:
# fixed size = 1024
tile_size = 512
factor = 2
train_zip,mask_zip = 'train.zip', 'masks.zip'

with zipfile.ZipFile(train_zip, 'w') as img_out,zipfile.ZipFile(mask_zip, 'w') as mask_out:
    for i in range(mask_df.shape[0]):
        row = mask_df.loc[i]
        index, encoding = row[0],row[1]
        #read image and generate the mask
        img = tiff.imread(os.path.join(train_folder,index+'.tiff'))
        if len(img.shape) == 5:
            img = np.transpose(img.squeeze(), (1,2,0))
        mask = rle2mask(encoding, (img.shape[1],img.shape[0]))
    
        ## padding images so that split can be correct
        shape = img.shape
        pad0 = (tile_size*factor - shape[0]%(tile_size*factor))%(tile_size*factor) 
        pad1 = (tile_size*factor - shape[1]%(tile_size*factor))%(tile_size*factor)
                           
        img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],constant_values=0)
        mask = np.pad(mask,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2]],constant_values=0)
    
        #split image and mask into tiles using the reshape+transpose trick
        img = cv2.resize(img,(img.shape[0]//factor,img.shape[1]//factor),interpolation = cv2.INTER_AREA)
        img = img.reshape(img.shape[0]//tile_size,tile_size,img.shape[1]//tile_size,tile_size,3)
        img = img.transpose(0,2,1,3,4).reshape(-1,tile_size,tile_size,3)

        mask = cv2.resize(mask,(mask.shape[0]//factor,mask.shape[1]//factor),interpolation = cv2.INTER_NEAREST)
        mask = mask.reshape(mask.shape[0]//tile_size,tile_size,mask.shape[1]//tile_size,tile_size)
        mask = mask.transpose(0,2,1,3).reshape(-1,tile_size,tile_size)

        #write data
        for i,(im,m) in enumerate(zip(img,mask)):
            if im.sum() == 0: continue
            im = cv2.imencode('.png',cv2.cvtColor(im, cv2.COLOR_RGB2BGR))[1]
            img_out.writestr(f'{index}_{i}.png', im)
            m = cv2.imencode('.png',m)[1]
            mask_out.writestr(f'{index}_{i}.png', m)

In [ ]:
!ls /kaggle/working